# git

In [ ]:
from pathlib import Path

import doit.tools
import importnb

try:
    import tomllib
except ImportError:
    pass

with importnb.Notebook():
    from jupyak.tasks import _actions as A
    from jupyak.tasks import _well_known as W
    from jupyak.tasks._yak import GitHub, Repo, Yak

In [ ]:
def yak_git_tasks(yak: Yak):
    if yak.lite.gist:
        work_path = yak.lite.gist_path
        gist_id = yak.lite.gist.split("/")[-1]
        url = f"https://gist.github.com/{gist_id}.git"
        name = "lite:gist"
        yield from init_task(name, url, work_path)
        local_refs, ref_paths, tasks = fetch_tasks(
            name=name, urls=[url], work_path=work_path, file_dep=[]
        )
        yield from tasks
        yield from merge_task(name, work_path, local_refs, ref_paths, [])

In [ ]:
def repo_git_tasks(repo: Repo):
    work_path, in_repo = repo.run_context
    gh: GitHub = repo.github
    git_config = work_path / W.GITCONFIG

    yield from init_task(repo.name, gh.url, work_path)

    local_refs, ref_paths, tasks = fetch_tasks(
        name=repo.name,
        urls=[gh.baseline, *gh.merge_with],
        work_path=work_path,
        file_dep=[],
    )

    yield from tasks

    targets = []
    if repo.js:
        targets += repo.js.package_jsons
    if repo.py:
        targets += [repo.work_path / ppt for ppt in repo.py.pyproject_tomls]

    yield from merge_task(repo.name, work_path, local_refs, ref_paths, targets)

In [ ]:
def init_task(name: str, url: str, work_path: Path):
    in_repo = {"cwd": work_path}
    git_config = work_path / W.GITCONFIG

    if not git_config.exists():
        init_actions = [
            (doit.tools.create_folder, [work_path]),
            A.git(["init", ".", "--initial-branch", W.WORK_BRANCH], in_repo),
            A.git(["remote", "add", "origin", url], in_repo),
        ]
    elif url not in git_config.read_text(encoding="utf-8"):
        init_actions = [
            A.git(["remote", "rm", "origin"], in_repo),
            A.git(["branch", "-D", "main"], in_repo),
            A.git(["remote", "add", "origin", url], in_repo),
        ]
    else:
        init_actions = []

    init_actions += [
        A.git(["status"], in_repo),
    ]

    yield dict(
        name=f"{name}:init",
        uptodate=[lambda: git_config.exists(), doit.tools.config_changed(url)],
        doc=f"> initialize an empty repo for {name}",
        actions=init_actions,
        targets=[git_config],
    )

In [ ]:
def merge_task(
    name: str,
    work_path: Path,
    local_refs: list[str],
    file_dep: list[Path],
    targets: list[Path],
):
    in_repo = {"cwd": work_path}
    targets += [work_path / ".git/refs/heads" / W.WORK_BRANCH]
    actions = []
    for ref in local_refs:
        if not actions:
            actions += [A.git(["checkout", "-fB", W.WORK_BRANCH, ref], in_repo)]
        else:
            actions += [A.git(["merge", ref, "-m", f"[jpyk]: merge {ref}"], in_repo)]

    yield dict(
        name=f"{name}:checkout",
        actions=actions,
        file_dep=file_dep,
        targets=targets,
    )

In [ ]:
def fetch_tasks(name: str, urls: str, work_path: Path, file_dep: list[Path]):
    in_repo = {"cwd": work_path}
    local_refs = []
    ref_paths = []
    tasks = []
    targets = []
    file_dep += [work_path / W.GITCONFIG]
    fetch = ["fetch", "--no-tags"]

    if len(urls) == 1:
        fetch += ["--depth=1"]

    for url in urls:
        remote_ref, local_ref, target = _url_to_refs_and_target(url, work_path)
        task = dict(
            name=f"{name}:fetch:{local_ref}",
            doc=f"> fetch {name} at `{remote_ref}` to `{local_ref}`",
            actions=[
                A.git([*fetch, "origin", f"{remote_ref}:{local_ref}"], in_repo),
            ],
            file_dep=file_dep,
            targets=[target],
        )
        tasks += [task]
        ref_paths += [target]
        local_refs += [local_ref]
    return local_refs, ref_paths, tasks

In [ ]:
def _url_to_refs_and_target(url: str, work_path: Path) -> tuple[str, str, Path]:
    dot_git = work_path / ".git"
    heads = dot_git / "refs/heads"
    tags = dot_git / "refs/tags"
    bits = url.split("#")[0].split("?")[0].split("/")
    if "tree" in bits:
        tree = "/".join(bits[bits.index("tree") + 1 :])
        return tree, tree, heads / tree
    if "pull" in bits:
        pr = bits[bits.index("pull") + 1]
        return f"pull/{pr}/head", f"pr-{pr}", heads / f"pr-{pr}"
    if "releases" in url and "tag" in url:
        tag = "/".join(bits[bits.index("tag") + 1 :])
        return tag, tag, tags / tag
    if "gist.github.com" in bits:
        return "main", "main", heads / "main"
    raise ValueError(f"Don't know how to work with {url}")